In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import transformers

import nltk
import re


from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
plt.style.use('seaborn')

In [ ]:
resumeDataSet = pd.read_csv('../input/resume-dataset/UpdatedResumeDataSet.csv' ,encoding='utf-8')
resumeDataSet['cleaned_resume'] = ''
resumeDataSet.head(20)

In [ ]:
print ("The distinct categories of resume -")
print (resumeDataSet['Category'].unique())
print (resumeDataSet['Category'].unique().shape)

In [ ]:
print (resumeDataSet['Category'].value_counts())

## Preprocessing

In [ ]:
import re
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)    
    resumeText = re.sub('#\S+', '', resumeText)  
    resumeText = re.sub('@\S+', '  ', resumeText)  
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  
    return resumeText

In [ ]:
resumeDataSet['cleaned_resume'] = resumeDataSet.Resume.apply(lambda x: cleanResume(x))


In [ ]:
print (resumeDataSet['cleaned_resume'][0])

## Transform target variable using label encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

var_mod = ['Category']
le = LabelEncoder()
for i in var_mod:
    resumeDataSet[i] = le.fit_transform(resumeDataSet[i])


## Text representation using TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

requiredText = resumeDataSet['cleaned_resume'].values
requiredTarget = resumeDataSet['Category'].values

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    stop_words='english',
    max_features=1500)
word_vectorizer.fit(requiredText)
WordFeatures = word_vectorizer.transform(requiredText)



In [ ]:
requiredTarget

## Split the date

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(WordFeatures,requiredTarget,random_state=0, test_size=0.2,stratify=requiredTarget)
print(X_train.shape)
print(X_test.shape)

In [ ]:
# import nltk
# from nltk.corpus import stopwords


In [ ]:
from xgboost  import XGBClassifier,plot_importance
from sklearn.metrics import accuracy_score

In [ ]:
xgb = XGBClassifier()  
xgb.fit(X_train, y_train)

In [ ]:
y_pred = xgb.predict(X_test)
# predictions = [round(value) for value in y_pred]

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

## Using our CVs to test the model

In [ ]:
Test_CVs = pd.read_csv('../input/test-cvs/Test_CVs.csv' ,encoding='utf-8')

In [ ]:
Test_CVs['cleaned_resume'] = ''
Test_CVs['cleaned_resume']= Test_CVs.Resume.apply(lambda x: cleanResume(x))

In [ ]:
Test_CVs

In [ ]:
print (Test_CVs['cleaned_resume'][0])

In [ ]:
test_text = Test_CVs['cleaned_resume'].values

# word_vectorizer = TfidfVectorizer(
#     sublinear_tf=True,
#     stop_words='english',
#     max_features=1500)
# word_vectorizer.fit(test_text)
WordFeatures = word_vectorizer.transform(test_text)

In [ ]:
WordFeatures.shape

In [ ]:
X_test2=WordFeatures

In [ ]:
y_pred2 = xgb.predict(X_test2)

In [ ]:
y_pred2

In [ ]:
prediction= le.inverse_transform(y_pred2)

In [ ]:
prediction

## Decision Tree

In [ ]:
from sklearn import tree

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [ ]:
y_pred_dt = clf.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred_dt)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
y_pred2_dt = clf.predict(X_test2)

In [ ]:
prediction_dt= le.inverse_transform(y_pred2_dt)

In [ ]:
prediction_dt

In [ ]:
# !pip install PyPDF2
# import PyPDF2

In [ ]:
xgb_auc = roc_auc_score(y_test, y_pred2)
dt_auc = roc_auc_score(y_test, y_pred2_dt)
# summarize scores
print('XGboost: ROC AUC=%.3f' % (xgb_auc))
print('Decision Tree: ROC AUC=%.3f' % (dt_auc))
# calculate roc curves
xgb_fpr, xgb_tpr, _ = roc_curve(y_test, y_pred2)
dt_fpr, dt_tpr, _ = roc_curve(y_test, y_pred2_dt)
# plot the roc curve for the model
pyplot.plot(xgb_fpr, xgb_tpr, linestyle='--', label='XGboost')
pyplot.plot(dt_fpr, dt_tpr, marker='.', label='Decision Tree')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()